In [25]:
val sqlC = new org.apache.spark.sql.SQLContext(sc)
import sqlC.implicits._
val spark2 = spark
import spark2.implicits._

In [5]:
val spark2 = spark
import spark2.implicits._

In [29]:
val spark2 = new org.apache.spark.sql.SparkSession(sc)

Name: Compile Error
Message: <console>:34: error: constructor SparkSession in class SparkSession cannot be accessed in class $iw
       val spark2 = new org.apache.spark.sql.SparkSession(sc)
                    ^
StackTrace: 

In [3]:
// RDD

val rdd = (sc.textFile("notebooks/data/people.csv")
    .map(_.split(",")))
rdd

MapPartitionsRDD[2] at map at <console>:29

In [11]:
// Dataset

val df = spark.read.json("notebooks/data/people.json")
df.select($"age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
|  26|
|  15|
+----+



In [19]:
import sys.process._

In [20]:
"more notebooks/data/people.json" !

{"name":"Michael"}
{"name":"Andy", "age":30}
{"name":"Justin", "age":19}
{"name":"Justin", "age":26}
{"name":"Justin", "age":15}


In [23]:
case class Person(name: String, age: Option[BigInt])
val ds = spark2.read.json("notebooks/data/people.json").as[Person]
ds.select($"age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
|  26|
|  15|
+----+



In [30]:
ds.map(_.age.getOrElse(0)).sum()

Name: Compile Error
Message: <console>:49: error: Unable to find encoder for type stored in a Dataset.  Primitive types (Int, String, etc) and Product types (case classes) are supported by importing spark.implicits._  Support for serializing other types will be added in future releases.
       ds.map(_.age.getOrElse(0)).sum()
             ^
StackTrace: 

In [2]:
val dataset = (0 to 9).toDS
dataset.filter(_ % 2 == 0).count

5

In [5]:
spark.range(10).filter('id === 0).explain(true)

== Parsed Logical Plan ==
'Filter ('id = 0)
+- Range (0, 10, splits=2)

== Analyzed Logical Plan ==
id: bigint
Filter (id#71L = cast(0 as bigint))
+- Range (0, 10, splits=2)

== Optimized Logical Plan ==
Filter (id#71L = 0)
+- Range (0, 10, splits=2)

== Physical Plan ==
*Filter (id#71L = 0)
+- *Range (0, 10, splits=2)


In [8]:
val million = 0 until math.pow(10, 6).toInt
sc.parallelize(million).cache.count

1000000

In [9]:
million.toDF.cache.count

1000000

In [12]:
case class Foo(i: Int, s: String, d: Double)
val data = (0 to 10).map(x => Foo(x, x.toString, x * 3.14)).toDS

In [18]:
data.toJSON.show

+--------------------+
|               value|
+--------------------+
|{"i":0,"s":"0","d...|
|{"i":1,"s":"1","d...|
|{"i":2,"s":"2","d...|
|{"i":3,"s":"3","d...|
|{"i":4,"s":"4","d...|
|{"i":5,"s":"5","d...|
|{"i":6,"s":"6","d...|
|{"i":7,"s":"7","d...|
|{"i":8,"s":"8","d...|
|{"i":9,"s":"9","d...|
|{"i":10,"s":"10",...|
+--------------------+



# Comparison

In [4]:
val sqlC = new org.apache.spark.sql.SQLContext(sc)
import sqlC.implicits._

In [1]:
// RDD

val rdd = (sc.textFile("data/people.csv")
    .map(_.split(",")))
rdd

MapPartitionsRDD[2] at map at <console>:22

In [6]:
// Dataset

val df = spark.read.json("data/people.json")
df.select($"name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
| Justin|
| Justin|
+-------+



In [7]:
val dataset = (0 to 9).toDS
dataset.filter(_ % 2 == 0).count

5

In [ ]:
spark.range(10).filter('id === 0).explain(true)

## Memory Performance

Run these two jobs and goto localhost:4040

https://jaceklaskowski.gitbooks.io/mastering-apache-spark/content/spark-sql-tungsten.html

In [ ]:
val million = 0 until math.pow(10, 6).toInt
sc.parallelize(million).cache.count

In [ ]:
million.toDF.cache.count

## More Stuff

In [11]:
val df = (spark.read
  .option("header", true)
  .option("inferSchema", true)
  .json("data/people.json"))
  
df.groupBy("age").count()

[age: bigint, count: bigint]

# Reading data from Implied Schema

In [14]:
import org.apache.spark.sql.types._

val explicitSchema = StructType(
  StructField("name", StringType, false) ::
  StructField("city", StringType, true) ::
  StructField("country", StringType, true) ::
  StructField("age", IntegerType, true) :: Nil
)

val inExplicit = (spark.readStream
  .schema(explicitSchema)
  .format("csv")
  .option("header", true)
  .option("maxFilesPerTrigger", 1)
  .load("data/people.csv"))

In [15]:
val inImplied = (spark.read
  .format("csv")
  .option("header", true)
  .option("inferSchema", true)
  .load("data/people.csv"))
  
val impliedSchema = inImplied.schema

println(impliedSchema)
println(explicitSchema)

Name: Unknown
Message: Unable to retrieve error!
StackTrace: 